In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.models import load_model
import seaborn as sns
import cv2
import os

In [10]:
DATADIRTRAIN=r"C:\Users\vaibhav\Downloads\archive (11)\Sign Language for Numbers"
train=ImageDataGenerator(rescale=1/255,validation_split=0.2)


In [11]:
train_data= train.flow_from_directory(
       DATADIRTRAIN ,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',subset='training')

valid_data = train.flow_from_directory(
        DATADIRTRAIN,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',subset='validation')

Found 13200 images belonging to 11 classes.
Found 3300 images belonging to 11 classes.


In [12]:
train_data.class_indices

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'unknown': 10}

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(11, activation='softmax'))

model.summary()
#Convolution layer − It is the primary building block and perform computational tasks based on convolution function.
#Pooling layer − It is arranged next to convolution layer and is used to reduce the size of inputs by removing unnecessary information so computation can be performed faster.
#Fully connected layer − It is arranged to next to series of convolution and pooling layer and classify input into various categories.

'''
The dilation_rate parameter of the Conv2D class is a 2-tuple of integers, which controls the dilation rate for dilated convolution.
The Dilated Convolution is the basic convolution applied to the input volume with defined gaps.
We use this parameter when working with higher resolution images and fine-grained details are important
'''

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 32)        2432      
_________________________________________________________________
activation (Activation)      (None, 60, 60, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        3

'\nThe dilation_rate parameter of the Conv2D class is a 2-tuple of integers, which controls the dilation rate for dilated convolution.\nThe Dilated Convolution is the basic convolution applied to the input volume with defined gaps.\nWe use this parameter when working with higher resolution images and fine-grained details are important\n'

In [14]:
from keras import optimizers
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)
#Early stopping is a method that allows you to specify an arbitrary large number of training epochs and stop training once the model performance stops improving

In [28]:
model.fit(train_data,epochs=20,batch_size=32, validation_data = valid_data,callbacks=[early_stop])
#One Epoch is when an ENTIRE dataset is passed forward and backward through the neural network only ONCE

Epoch 1/20
413/413 [==============================] - 38s 93ms/step - loss: 0.0205 - accuracy: 0.9929 - val_loss: 0.8646 - val_accuracy: 0.8594
Epoch 2/20
413/413 [==============================] - 40s 96ms/step - loss: 0.0247 - accuracy: 0.9932 - val_loss: 0.5976 - val_accuracy: 0.8764
Epoch 3/20
413/413 [==============================] - 40s 96ms/step - loss: 0.0279 - accuracy: 0.9917 - val_loss: 0.8218 - val_accuracy: 0.8779
Epoch 4/20
413/413 [==============================] - 40s 97ms/step - loss: 0.0313 - accuracy: 0.9908 - val_loss: 0.6885 - val_accuracy: 0.8833


In [29]:
model.save(r'C:\Users\vaibhav\Downloads\archive (11)\Sign Language for Numbers\model1.h5')

In [2]:
model=load_model(r"C:\Users\vaibhav\Downloads\Numbers.h5")

In [21]:
background = None
accumulated_weight = 0.5

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350
def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [22]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    _ , thresholded = cv2.threshold(diff, threshold,255,cv2.THRESH_BINARY)
    # Grab the external contours for the image
    contours, hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    else:
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment_max_cont)

In [27]:
cam = cv2.VideoCapture(0)
num_frames =0
while True:
    ret, frame = cam.read()

    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    #In image processing, a Gaussian blur is the result of blurring an image by a Gaussian function.


    if num_frames < 70:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT",
  (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right,
      ROI_top)], -1, (255, 0, 0),1)
            
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            thresholded = cv2.resize(thresholded, (64, 64))
            thresholded = cv2.cvtColor(thresholded,cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded,(1,thresholded.shape[0],thresholded.shape[1],3))
            
            #thresholded = cv2.cvtColor(thresholded, cv2.COLOR_RGB2GRAY)
           #print(thresholded.shape)
            #cv2.imshow("Thesholded Hand Image", thresholded[:3])
            
            
            
            
            pred = model.predict(thresholded)
            
            
            
            #Added manually down one
            
            
            letter=""
            for val in range(0,len(pred[0])):
                if(pred[0][val]==1):
            
                    if val<26:
                        letter=str(val)
                    else:
                        if val==26:
                            letter="EMPTY"
                        elif val==27:
                            letter="SPACE"
                        else:
                            letter="Model was not able to predict"
            
            
            
            
            
            
            
            
            
            
            
            
            #cv2.putText(frame_copy, word_dict[np.argmax(pred)],(170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.putText(frame_copy, letter,(170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,
    ROI_bottom), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "Word _ _ _",(10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()